In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import re

Preview dataset


In [2]:
from google.colab import files


uploaded = files.upload()


Saving imdb_reviews.csv to imdb_reviews.csv


In [3]:
data = pd.read_csv('imdb_reviews.csv')

print(data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


In [10]:
english_stops = set(stopwords.words('english'))

**Load and Clean Dataset**

In the original dataset, the reviews are still dirty. There are still html tags, numbers, uppercase, and punctuations. This will not be good for training, so in load_dataset() function, beside loading the dataset using pandas, I also pre-process the reviews by removing html tags, non alphabet (punctuations and numbers), stop words, and lower case all of the reviews.

In [11]:
def load_dataset():
    df = pd.read_csv('imdb_reviews.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case

    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Reviews
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object 

Sentiment
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


Split Dataset

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
48052    [i, love, don, knotts, let, say, front, he, en...
22663    [in, film, blow, up, antonioni, hero, question...
45296    [woof, pretty, boring, might, well, shot, blac...
13735    [i, checked, movie, based, favorable, review, ...
321      [i, watched, movie, countless, times, never, f...
                               ...                        
11441    [mild, spoilers, with, exception, the, termina...
38754    [you, lived, japan, awhile, enjoy, beauty, mov...
17944    [i, found, charming, adaptation, lively, full,...
29520    [as, spiritualist, non, christian, i, thought,...
627      [why, canada, turn, decent, good, movies, ever...
Name: review, Length: 40000, dtype: object 

49309    [did, writers, pay, people, come, write, posit...
39185    [wolfgang, peterson, directs, thriller, clint,...
872      [this, second, hitchcock, film, appear, list, ...
7783     [because, writers, guild, america, strike, sho...
20441    [a, great, storyline, message, joan, plowright...
 

In [13]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

**Tokenize and Pad/Truncate Reviews**
A Neural Network only accepts numeric data, so we need to encode the reviews. I use tensorflow.keras.preprocessing.text.Tokenizer to encode the reviews into integers, where each unique word is automatically indexed (using fit_on_texts method) based on x_train.
x_train and x_test is converted into integers using texts_to_sequences method.

Each reviews has a different length, so we need to add padding (by adding 0) or truncating the words to the same length (in this case, it is the mean of all reviews length) using tensorflow.keras.preprocessing.sequence.pad_sequences.

post, pad or truncate the words in the back of a sentence
pre, pad or truncate the words in front of a sentence

In [14]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[    1    41   346 ...  2901   389    66]
 [   49     4  2133 ...  4165 10197  4918]
 [25102    93   250 ... 17096   284    87]
 ...
 [    1   162  1236 ...     0     0     0]
 [  109 42126   583 ...     0     0     0]
 [  359  3736   371 ...     0     0     0]] 

Encoded X Test
 [[ 1445   838   871 ...     0     0     0]
 [15608  9013  4464 ...     0     0     0]
 [    8   245  1355 ...   175   150   707]
 ...
 [    2   249   279 ...     0     0     0]
 [   23 52988   259 ...     0     0     0]
 [    2   800    14 ... 16033  2355  1020]] 

Maximum review length:  130


**Build Architecture/Model**

**Embedding Layer:** in simple terms, it creates word vectors of each word in the word_index and group words that are related or have similar meaning by analyzing other words around them.

**LSTM Layer: **to make a decision to keep or throw away data by considering the current input, previous output, and previous memory. There are some important components in LSTM.

Forget Gate, decides information is to be kept or thrown away
Input Gate, updates cell state by passing previous output and current input into sigmoid activation function
Cell State, calculate new cell state, it is multiplied by forget vector (drop value if multiplied by a near 0), add it with the output from input gate to update the cell state value.
Ouput Gate, decides the next hidden state and used for predictions

In [15]:
# ARCHITECTURE
EMBED_DIM = 32
LSTM_OUT = 64

model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(LSTM(LSTM_OUT))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 130, 32)           2955840   
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 2980737 (11.37 MB)
Trainable params: 2980737 (11.37 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None



Training

In [16]:
checkpoint = ModelCheckpoint(
    'models/LSTM.h5',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)

In [17]:
model.fit(x_train, y_train, batch_size = 128, epochs = 5, callbacks=[checkpoint])

Epoch 1/5
313/313 [==============================] - ETA: 0s - loss: 0.4540 - accuracy: 0.7566
Epoch 1: accuracy improved from -inf to 0.75660, saving model to models/LSTM.h5
313/313 [==============================] - 75s 229ms/step - loss: 0.4540 - accuracy: 0.7566
Epoch 2/5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


313/313 [==============================] - ETA: 0s - loss: 0.2112 - accuracy: 0.9237
Epoch 2: accuracy improved from 0.75660 to 0.92375, saving model to models/LSTM.h5
313/313 [==============================] - 68s 218ms/step - loss: 0.2112 - accuracy: 0.9237
Epoch 3/5
313/313 [==============================] - ETA: 0s - loss: 0.1229 - accuracy: 0.9625
Epoch 3: accuracy improved from 0.92375 to 0.96245, saving model to models/LSTM.h5
313/313 [==============================] - 69s 221ms/step - loss: 0.1229 - accuracy: 0.9625
Epoch 4/5
313/313 [==============================] - ETA: 0s - loss: 0.0904 - accuracy: 0.9738
Epoch 4: accuracy improved from 0.96245 to 0.97378, saving model to models/LSTM.h5
313/313 [==============================] - 70s 224ms/step - loss: 0.0904 - accuracy: 0.9738
Epoch 5/5
313/313 [==============================] - ETA: 0s - loss: 0.0602 - accuracy: 0.9834
Epoch 5: accuracy improved from 0.97378 to 0.98343, saving model to models/LSTM.h5
313/313 [=============

Testing

In [23]:
y_pred = (model.predict(x_test) > 0.5).astype("int64")
true = 0
for i, y in enumerate(y_test):
    if y == y_pred[i]:
        true += 1

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred) - true))
print('Accuracy: {}'.format(true/len(y_pred)*100))

313/313 [==============================] - 8s 25ms/step
Correct Prediction: 8755
Wrong Prediction: 1245
Accuracy: 87.55
